<a href="https://colab.research.google.com/github/brucker3/KDM_spring2021/blob/main/ICP5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# intsall spark and importing necessry packages 
!pip install pyspark
from __future__ import print_function
from pyspark import SparkConf, SparkContext
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql import SparkSession
from pyspark.ml.feature import NGram
from pyspark.ml.feature import Word2Vec

# creating spark session
spark = SparkSession.builder.appName("ICP5").getOrCreate()




     |████████████████████████████████| 204.8MB 39kB/s 
     |████████████████████████████████| 204kB 48.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186687 sha256=2ae64b1d956d9cd1eadc2c9e876761d250068050fa8a1fa8c85e7f33c1462f38
  Stored in directory: /root/.cache/pip/wheels/8b/09/da/c1f2859bcc86375dc972c5b6af4881b3603269bcc4c9be5d16
Successfully built pyspark


In [51]:
# https://sparkbyexamples.com/spark/spark-read-text-file-rdd-dataframe/
# https://colab.research.google.com/github/computationalcore/introduction-to-python/blob/master/notebooks/4-files/PY0101EN-4-1-ReadFile.ipynb#scrollTo=ArrqzEmQx2_M
# read in of all 5 txt files 
# I tried to do this a cleaner way but spark kept skipping over some of the files 
txt1 = "../content/txt1.txt"
txt2 = "../content/txt2.txt"
txt3 = "../content/txt3.txt"
txt4 = "../content/txt4.txt"
txt5 = "../content/txt5.txt"
# opening and converting read in files to use able form 
txt1 = open(txt1, "r")
txt2 = open(txt2, "r")
txt3 = open(txt3, "r")
txt4 = open(txt4, "r")
txt5 = open(txt5, "r")

# creating spark dataframe wiht the input data. 
sentenceData = spark.createDataFrame([
        (0, txt1.read()),
        (1, txt2.read()),
        (2, txt3.read()),
        (3, txt4.read()),
        (4, txt5.read())
    ], ["label", "sentence"])

# cheking data type and that df is set up corretly 
type(sentenceData)
sentenceData.head(5)


[Row(label=0, sentence='When you ask Americans a binary question about reopening schools, support for reopening appears high. For example, a Jan. 28-30 poll by RMG Research for Just the News found that 53 percent of registered voters thought schools in their area should be open for in-person learning, while just 31 percent thought they should remain closed. And according to a Feb. 12-15 Morning Consult/Politico survey, 57 percent of registered voters thought it was a good idea for K-12 schools to reopen in fall 2021, while just 26 percent thought it was a bad idea.\n\nAdd an option for hybrid learning, however, and the nation is more ambivalent. According to a YouGov/HuffPost poll conducted Feb. 3-7, adults were evenly divided among the three options. Twenty-seven percent thought schools should be completely reopened, 29 percent thought schools should be partially reopened and 30 percent thought they should be closed or online-only.\n'),
 Row(label=1, sentence="So a new coach is in. Th

In [ ]:
# tokenizer to set up for IDF calculation 
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)



# applying tf on the words data
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors


# calculating the IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)


#displaying the results
rescaledData.select("label", "features").show()